<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Product_Match_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymssql
!pip install rapidfuzz
import pymssql
import pandas as pd
import numpy as np
from google.colab import auth
from google.auth import default
import re
from rapidfuzz import fuzz, process
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive

#Mount Google Drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.6 MB/s eta 0:00:00
Mounted at /content/drive


# Data

In [ ]:
connection = pymssql.connect(server='35.187.243.174', user='tim_powerbi',
                             password='dGltcG93ZXJiaQ', database='mkt_insight', port = '1433')

query_2 = """
SELECT [Kode Produk PI], [Nama Produk], Procode, Prodesc FROM mkt_insight.dbo.ListAllProductComplete
GROUP BY [Kode Produk PI], [Nama Produk], Procode, Prodesc"""

data_produk_pi = pd.DataFrame(pd.read_sql_query(query_2, connection))

data_produk_pi

,Kode Produk PI,Nama Produk,Procode,Prodesc
0,000058,FAKTU OINTMENT 20 G,0101183,FAKTU OINT 20GR
1,000069,FAKTU SUPPOSITORIA,0101184,FAKTU SUPP 10`S
2,000070,NEBACETIN OINTMENT 5 G,0102055,NEBACETIN OINT 5GR
3,000080,NEBACETIN POWDER 5 G,0102056,NEBACETIN POWDER 5GR
4,000102,ALBOTHYL GEL 50 G,0100061,ALBOTHYL 18MG GEL 50GR
...,...,...,...,...
1235,0514363,ILLUMINARE PORE CLEAR FACIAL WASH 100G,0514363,ILLUMINARE PORE CLEAR FACIAL WASH 100G
1236,3000854,ILLUMINARE YOUTH ACTIVATOR NOURISH SERUM 30 ML,3000854,ILLUMINARE YOUTH ACTIVATOR NOURISH SERUM 30 ML
1237,3000855,ILLUMINARE PORE SERUM 30ML,3000855,ILLUMINARE PORE SERUM 30ML
1238,3002105,ILLUMINARE SUN PROTECT FACE & BODY LOTION SPF5...,3002105,ILLUMINARE SUN PROTECT FACE & BODY LOTION SPF5...


In [ ]:
data_ims = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/IMS Compiled Q3 2019 - Q2 2024 (List Product).xlsx', sheet_name='Sheet1')
data_survey = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Hospital Survey Product Name Pivot.xlsx', sheet_name='Sheet1')

In [ ]:
data_prdk_survey = data_survey.copy()
data_ims_prdk = data_ims.copy()
data_pi_prdk = data_produk_pi[['Kode Produk PI', 'Nama Produk']].copy()

# Cleaning Data

In [ ]:
import re
import pandas as pd

def clean_product_name(name):
    return name.split(' -')[0].lower()

def clean_name_survey(name):
    # Hapus karakter khusus tetapi tetap menjaga format yang benar
    name = re.sub(r'[\[\]{}@]', '', name)  # Hapus tanda [], {}, dan @
    name = re.sub(r'(?<!\s)-(?<!\s)', ' ', name)  # Ganti "-" yang tidak diapit spasi dengan spasi
    name = re.sub(r'[()]', '', name)  # Hapus tanda kurung biasa tetapi biarkan isinya

    # Hilangkan spasi ganda yang mungkin muncul akibat penghapusan
    name = re.sub(r'\s+', ' ', name).strip()

    return name.lower()

def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\btabs?\b|\btablets?\b|\btablet\b': 'tab',
        r'\bcaps?\b|\bcapsules?\b|\bcapsul\b': 'cap',
        r'\bkapsul\b|\bkapsule\b|\bkap\b': 'cap',
        r'\bsuspensi\b|\bsuspension\b|\bsusp\b': 'susp',
        r'\bserbuk\b|\bbubuk\b|\bpowder\b|\bpwd\b': 'pwd',
        r'\boint\b|\bointment\b|\bsalep\b': 'oint',
        r'\bkaplet\b|\bcaplets?\b|\bkapl\b|\bcaplet\b': 'cap',
        r'\binj\b|\binjeksi\b|\binjection\b': 'inj',
        r'\bamp\b|\bampule\b|\bampul\b': 'amp',
        r'\bcream\b|\bcrm\b|\bkrim\b|\bcr\b': 'crm',
        r'\bsupp\b|\bsuppositoria\b|\bsuppos\b': 'suppos',
        r'\binf\b|\binfusion\b|\binfus\b': 'inf',
        r'\bsyrup\b|\bsirup\b|\bsyr\b': 'syr',
        r'\bdrop\b|\bdrops\b|\btetes\b': 'drp',
        r'\bconc\b|\bconcentrate\b': 'conc',
        r'\bsol\b|\bsolution\b': 'sol',
        r'\bgel\b': 'gel'
    }

    # Pastikan input berupa string
    if not isinstance(product_name, str) or product_name.strip() == '':
        return ''

    # Mengganti bentuk sediaan dalam nama produk
    for pattern, replacement in dosage_form_map.items():
        product_name = re.sub(pattern, replacement, product_name, flags=re.IGNORECASE)

    return product_name.upper()  # Kembalikan hasil dalam huruf besar

data_ims_prdk['Cleaned Brand Name'] = data_ims_prdk['Product'].apply(clean_product_name)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Product Name'].apply(clean_name_survey)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(extract_dosage_form)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].str.upper()
data_pi_prdk['Cleaned Name'] = data_pi_prdk['Nama Produk'].apply(extract_dosage_form)

In [ ]:
data_pi_prdk

,Kode Produk PI,Nama Produk,Cleaned Name
0,000058,FAKTU OINTMENT 20 G,FAKTU OINT 20 G
1,000069,FAKTU SUPPOSITORIA,FAKTU SUPPOS
2,000070,NEBACETIN OINTMENT 5 G,NEBACETIN OINT 5 G
3,000080,NEBACETIN POWDER 5 G,NEBACETIN PWD 5 G
4,000102,ALBOTHYL GEL 50 G,ALBOTHYL GEL 50 G
...,...,...,...
1235,0514363,ILLUMINARE PORE CLEAR FACIAL WASH 100G,ILLUMINARE PORE CLEAR FACIAL WASH 100G
1236,3000854,ILLUMINARE YOUTH ACTIVATOR NOURISH SERUM 30 ML,ILLUMINARE YOUTH ACTIVATOR NOURISH SERUM 30 ML
1237,3000855,ILLUMINARE PORE SERUM 30ML,ILLUMINARE PORE SERUM 30ML
1238,3002105,ILLUMINARE SUN PROTECT FACE & BODY LOTION SPF5...,ILLUMINARE SUN PROTECT FACE & BODY LOTION SPF5...


# FIXED

## Data IMS and PI Product

In [ ]:
# Fungsi untuk mengekstrak angka dan unit dari nama produk
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z]*)', name)  # Tangkap angka + unit
    return {f"{num} {unit}".strip() for num, unit in matches}  # Gunakan set agar unik

# Fungsi untuk mencocokkan angka dengan unit
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0  # Jika salah satu kosong, skor = 0

    survey_set = set(survey_numbers)
    pack_set = set(pack_numbers)
    intersection = survey_set.intersection(pack_set)

    # Skor dihitung berdasarkan banyaknya pasangan angka + satuan yang cocok
    score = (len(intersection) / max(len(survey_set), len(pack_set))) * 100
    return score

# Fungsi untuk mencocokkan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CAP",
                    "CRM", "SUSP", "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    # Temukan bentuk sediaan dalam kedua string
    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    # Jika bentuk sediaan ada di kedua produk, gunakan fuzzy matching
    if survey_form and ims_form:
        return fuzz.partial_ratio(survey_form, ims_form)

    return 50  # Jika tidak ada bentuk sediaan, beri skor default rendah

def calculate_similarity(cleaned_survey_name, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)  # Pastikan string

    for pack_tuple in pack_list:
        pack_name, molecule = pack_tuple  # Unpack tuple lebih awal
        pack_numbers = extract_numbers_and_units(pack_name)  # Sekarang ini string

        # Hitung skor kemiripan angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)

        # Hitung skor kemiripan string (nama produk)
        score_strings = fuzz.token_sort_ratio(cleaned_survey_name, pack_name)

        # Hitung kemiripan bentuk sediaan
        score_dosage_form = match_dosage_form(cleaned_survey_name, pack_name)

        # Gabungkan skor dengan bobot (angka lebih penting)
        combined_score = (0.6 * score_numbers) + (0.3 * score_strings) + (0.1 * score_dosage_form)
        results.append((pack_name, molecule, combined_score))

    return results

# Fungsi utama untuk memproses data dengan filter Cleaned Brand
def process_brand_matching(data_ims, data_survey):
    results = []

    # Ambil daftar nilai unik dari 'Cleaned Brand Name' di IMS
    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        # Gunakan regex untuk filter survey yang mengandung brand_keyword
        filtered_survey = data_survey[
            data_survey['Nama Produk'].str.contains(fr"\b{re.escape(brand_keyword)}\b", case=False, na=False)
        ]

        # Ambil hanya produk IMS dengan brand yang sama
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Nama Produk']
            kode_pi = survey_row['Kode Produk PI']

            # Menghitung skor kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list)

            # Cari produk IMS dengan skor tertinggi
            if similarity_scores:
                best_match_pack, best_molecule, best_score = max(similarity_scores, key=lambda x: x[2])

                # Simpan hasil terbaik
                results.append({
                    'Kode PI' : kode_pi,
                    'Product PI': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    "Pack Molecule String": best_molecule,
                    'Best Similarity Score': best_score
                })

    # Buat DataFrame hasil
    results_df = pd.DataFrame(results)
    return results_df.sort_values(by='Best Similarity Score', ascending=False)

# Contoh penggunaan
filtered_results = process_brand_matching(data_ims_prdk, data_pi_prdk)
filtered_results


,Kode PI,Product PI,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
558,010618,TAXILAN 500MG/5ML SUSPENSI 180ML,TAXILAN SUSP 500MG/5ML 180ML,SUCRALFATE,100.000000
556,014028,STOP-X CREAM 15G,STOP-X CRM 15G,CAMPHOR+EUGENOL+MENTHOL+NICOTINIC ACID+SALICYL...,100.000000
448,000388,POLYSILANE SUSP 180 ML,POLYSILANE SUSP 180ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
451,004909,POLYSILANE SUSPENSI 100 ML,POLYSILANE SUSP 100ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
452,005085,POLYSILANE SUSPENSI 180 ML,POLYSILANE SUSP 180ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
...,...,...,...,...,...
102,013600,BESTRAM 50 MG/ML INJECTION,BESTRAM AMP 100MG/2ML 2ML 5,TRAMADOL,11.489362
123,005822,CETOROS 1 G/5 ML INJEKSI,CETOROS AMP 200MG/ML 15ML 4,PIRACETAM,11.489362
193,012851,ETHIROS 175 MG/5 ML DRY SYRUP 60 ML,ETHIROS CAP 300MG 20 BLIS,ERDOSTEINE,11.379310
228,007373,HIPNOZ 15 MG/3 ML INJEKSI,HIPNOZ AMP 5MG/5ML 5ML 5,MIDAZOLAM,10.666667


In [ ]:
hasil = filtered_results.copy()
hasil = hasil.drop_duplicates(subset=['Product PI'])
hasil

,Kode PI,Product PI,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
558,010618,TAXILAN 500MG/5ML SUSPENSI 180ML,TAXILAN SUSP 500MG/5ML 180ML,SUCRALFATE,100.000000
556,014028,STOP-X CREAM 15G,STOP-X CRM 15G,CAMPHOR+EUGENOL+MENTHOL+NICOTINIC ACID+SALICYL...,100.000000
448,000388,POLYSILANE SUSP 180 ML,POLYSILANE SUSP 180ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
451,004909,POLYSILANE SUSPENSI 100 ML,POLYSILANE SUSP 100ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
452,005085,POLYSILANE SUSPENSI 180 ML,POLYSILANE SUSP 180ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
...,...,...,...,...,...
102,013600,BESTRAM 50 MG/ML INJECTION,BESTRAM AMP 100MG/2ML 2ML 5,TRAMADOL,11.489362
123,005822,CETOROS 1 G/5 ML INJEKSI,CETOROS AMP 200MG/ML 15ML 4,PIRACETAM,11.489362
193,012851,ETHIROS 175 MG/5 ML DRY SYRUP 60 ML,ETHIROS CAP 300MG 20 BLIS,ERDOSTEINE,11.379310
228,007373,HIPNOZ 15 MG/3 ML INJEKSI,HIPNOZ AMP 5MG/5ML 5ML 5,MIDAZOLAM,10.666667


In [ ]:
data_final_pi = data_produk_pi[['Kode Produk PI','Nama Produk']].merge(hasil,
                                                                    left_on='Kode Produk PI',
                                                                    right_on='Kode PI',
                                                                    how='left')
data_final_pi = data_final_pi[['Kode Produk PI','Nama Produk','Best Match IMS Pack','Best Similarity Score','Pack Molecule String']]

In [ ]:
import os

file_name = f"24022025 Data Hasil Match Product (PI and IMS).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_pi.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/24022025 Data Hasil Match Product (PI and IMS).xlsx


## Data Survey and IMS

In [ ]:
import pandas as pd
import re
from rapidfuzz import fuzz

# Fungsi untuk mengekstrak angka dan unit dari nama produk
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z]*)', name)  # Tangkap angka + unit
    return {f"{num} {unit}".strip() for num, unit in matches}  # Gunakan set agar unik

# Fungsi untuk mencocokkan angka dengan unit
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0  # Jika salah satu kosong, skor = 0

    survey_set = set(survey_numbers)
    pack_set = set(pack_numbers)
    intersection = survey_set.intersection(pack_set)

    # Skor dihitung berdasarkan banyaknya pasangan angka + satuan yang cocok
    score = (len(intersection) / max(len(survey_set), len(pack_set))) * 100
    return score

# Fungsi untuk mencocokkan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CAP",
                    "CRM", "SUSP", "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    # Temukan bentuk sediaan dalam kedua string
    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    # Jika bentuk sediaan ada di kedua produk, gunakan fuzzy matching
    if survey_form and ims_form:
        return fuzz.partial_ratio(survey_form, ims_form)

    return 50  # Jika tidak ada bentuk sediaan, beri skor default rendah

# Fungsi untuk menghitung skor kemiripan dengan bobot yang lebih baik
def calculate_similarity(cleaned_survey_name, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)  # Pastikan string

    for pack_tuple in pack_list:
        pack_name, molecule = pack_tuple  # Unpack tuple lebih awal
        pack_numbers = extract_numbers_and_units(pack_name)  # Sekarang ini string

        # Hitung skor kemiripan angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)

        # Hitung skor kemiripan string (nama produk)
        score_strings = fuzz.token_sort_ratio(cleaned_survey_name, pack_name)

        # Hitung kemiripan bentuk sediaan
        score_dosage_form = match_dosage_form(cleaned_survey_name, pack_name)

        # Gabungkan skor dengan bobot (angka lebih penting)
        combined_score = (0.6 * score_numbers) + (0.3 * score_strings) + (0.1 * score_dosage_form)
        results.append((pack_name, molecule, combined_score))

    return results


# Fungsi utama untuk memproses data dengan filter Cleaned Brand
def process_brand_matching(data_ims, data_survey):
    results = []
    # Ambil daftar nilai unik dari 'Cleaned Brand Name' di IMS
    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        # Gunakan regex untuk filter survey yang mengandung brand_keyword
        filtered_survey = data_survey[
            data_survey['Product Name'].str.contains(fr"\b{re.escape(brand_keyword)}\b", case=False, na=False)
        ]

        # Ambil hanya produk IMS dengan brand yang sama
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Product Name']
            kode_survey = survey_row['Kode']

            # Ambil pack list dari IMS yang memiliki brand yang sama
            pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String']))

            # Menghitung skor kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list)

            # Cari produk IMS dengan skor tertinggi
            if similarity_scores:
                best_match_pack, best_molecule, best_score = max(similarity_scores, key=lambda x: x[2])

                # Simpan hasil terbaik
                results.append({
                    'Kode Survey':kode_survey,
                    'Product Survey': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Pack Molecule String' : best_molecule,
                    'Best Similarity Score': best_score
                })

    # Buat DataFrame hasil
    results_df = pd.DataFrame(results)
    return results_df.sort_values(by='Best Similarity Score', ascending=False)

# Contoh penggunaan
filtered_results_survey = process_brand_matching(data_ims_prdk, data_prdk_survey)
filtered_results_survey

,Kode Survey,Product Survey,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
13242,49336,VOMETA 5MG/ML-10ML DROPS,VOMETA DRP 5MG/ML 10ML,DOMPERIDONE,100.000000
25009,45272,TB Vit 6 Syr 225ml,TB VIT 6 SYR 225ML,ISONIAZID,100.000000
23811,24017,LAPIFED EXP SYR 60ML,LAPIFED EXP SYR 60ML,GUAIFENESIN+PSEUDOEPHEDRINE+TRIPROLIDINE,100.000000
35592,24294,LAXOBERON DROPS 10ML,LAXOBERON DRP 10ML,PICOSULFURIC ACID,100.000000
35591,24293,Laxoberon drop 10ml,LAXOBERON DRP 10ML,PICOSULFURIC ACID,100.000000
...,...,...,...,...,...
27870,28765,N-EPI INJEKSI,N-EPI INFUSION AMP 1MG/ML 4ML 5,NOREPINEPHRINE,9.000000
6327,32869,"POLIDEMISIN EYE DROP 5 ML (DEXAMETHASONE, NEOM...",NEOMYCIN TAB 250MG 10 8,NEOMYCIN,8.842105
18300,8071,Channa Plus DS Syr,CHANNA CAP 500MG 30 BTL,CHANNA STRIATA,8.780488
6323,19250,"IH OTOPAIN EAR DROP (POLYMIXIN BSO4 , NEOMYCIN...",NEOMYCIN TAB 250MG 10 8,NEOMYCIN,7.636364


In [ ]:
data_prdk_survey.head()

,Kode,Product Name,Cleaned Name
0,1,ATORVASTATIN 20MG TAB/AMAROX,ATORVASTATIN 20MG TAB/AMAROX
1,2,BISOPROLOL 5 MG /HJ,BISOPROLOL 5 MG /HJ
2,3,BISOPROLOL 5 MG TABLET,BISOPROLOL 5 MG TAB
3,4,CALOS KAPLET,CALOS CAP
4,5,CAPTOPRIL 12.5 MG,CAPTOPRIL 12.5 MG


In [ ]:
data_final_survey = data_prdk_survey[['Kode','Product Name']].merge(filtered_results_survey[['Kode Survey','Best Match IMS Pack','Pack Molecule String','Best Similarity Score']],
                                                                    left_on='Kode',
                                                                    right_on='Kode Survey',
                                                                    how='left')
data_final_survey = data_final_survey[['Kode','Product Name','Best Match IMS Pack','Pack Molecule String','Best Similarity Score']]
data_final_survey

,Kode,Product Name,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
0,1,ATORVASTATIN 20MG TAB/AMAROX,ATORVASTATIN TAB FLM CTD 20MG 30,ATORVASTATIN,61.000000
1,2,BISOPROLOL 5 MG /HJ,BISOPROLOL TAB 5MG 30,BISOPROLOL,56.000000
2,3,BISOPROLOL 5 MG TABLET,BISOPROLOL TAB 5MG 30,BISOPROLOL,64.000000
3,4,CALOS KAPLET,CALOS TAB CHEW 500MG 30,CALCIUM,19.000000
4,5,CAPTOPRIL 12.5 MG,CAPTOPRIL TAB 12.5MG 50,CAPTOPRIL,56.500000
...,...,...,...,...,...
54492,50331,ZYVOX 600MG INJ 300ML,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,51.595745
54493,50332,ZYVOX 600MG TAB,ZYVOX TAB 600MG 10,LINEZOLID,67.272727
54494,50333,ZYVOX 600MG/300ML POLYBAG,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,60.882353
54495,50334,Zyvox Infusion 2 mg/mL - 300 mL,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,82.307692


In [ ]:
import os

file_name = f"24022025 Data Hasil Match Product (IMS & Survey).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_survey.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/24022025 Data Hasil Match Product (IMS & Survey).xlsx


# Database Data Match

## PI and IMS

In [ ]:
data_match_pi = pd.read_excel("/content/drive/MyDrive/Dataset Hasil Match Product/Data Hasil Match Product (PI and IMS).xlsx", sheet_name="Sheet1")
data_match_pi.head()

,Nama Produk (PI),Kata Pertama,Nama Produk (IMS),Skor Kemiripan
0,TAXILAN 500MG/5ML SUSPENSI 180ML,TAXILAN,TAXILAN SUSP 500MG/5ML 180ML,90.0
1,PROTOCORT 0.05 % CREAM 10 G,PROTOCORT,PROTOCORT CRM 0.05% 10G,85.8
2,HERCUM 0.05 % CREAM 10 G,HERCUM,HERCUM CRM 0.05% 10G,85.2
3,DEXOSYN 0.25 % CREAM 5 G,DEXOSYN,DEXOSYN CRM 0.25% 5G,85.2
4,NARFOZ 4 MG/5 ML SYRUP 30 ML,NARFOZ,NARFOZ SYR 4MG/5ML 30ML,79.2


In [ ]:
data_ims_2 = data_ims.copy()
data_match_pi["Nama Produk (IMS)"] = data_match_pi["Nama Produk (IMS)"].str.lower().str.strip()
data_ims_2["Pack"] = data_ims_2["Pack"].str.lower().str.strip()

kde_produk_pi = dict(zip(data_produk_pi["Nama Produk"], data_produk_pi["Item Kode"]))

database_match_pi = data_match_pi.merge(data_ims_2,
                                       left_on = "Nama Produk (IMS)",
                                       right_on = "Pack",
                                       how = 'left')
database_match_pi = database_match_pi[['Nama Produk (PI)','NFC1', "NFC1 Code","Pack Size","Pack Strength","Pack Volume","Pack Molecule String"]]
database_match_pi['KodePI'] = database_match_pi["Nama Produk (PI)"].map(kde_produk_pi)

# Reorder agar "Kode Produk" menjadi kolom pertama
kolom_urut = ["KodePI"] + [col for col in database_match_pi.columns if col != "KodePI"]
database_match_pi = database_match_pi[kolom_urut]


database_match_pi

,KodePI,Nama Produk (PI),NFC1,NFC1 Code,Pack Size,Pack Strength,Pack Volume,Pack Molecule String
0,010618,TAXILAN 500MG/5ML SUSPENSI 180ML,D - ORAL LIQUID ORDINARY,D,1.0,500MG,180ML,SUCRALFATE
1,009331,PROTOCORT 0.05 % CREAM 10 G,M - TOPIC DERMATOL HAEMOR EXTERN,M,1.0,0.05%,10G,BETAMETHASONE
2,009100,HERCUM 0.05 % CREAM 10 G,M - TOPIC DERMATOL HAEMOR EXTERN,M,1.0,0.05%,10G,CLOBETASOL
3,009276,DEXOSYN 0.25 % CREAM 5 G,M - TOPIC DERMATOL HAEMOR EXTERN,M,1.0,0.25%,5G,DESOXIMETASONE
4,015942,NARFOZ 4 MG/5 ML SYRUP 30 ML,D - ORAL LIQUID ORDINARY,D,1.0,4MG,30ML,ONDANSETRON
...,...,...,...,...,...,...,...,...
500,030222,COFITKLIN SPRAY 100 ML,NaN,NaN,NaN,NaN,NaN,NaN
501,030233,"NBC ACNE FACIAL FOAM, ACNE SERIES 100 ML",NaN,NaN,NaN,NaN,NaN,NaN
502,030277,MIJU FAIRY WHITE PUDDING SLEEPING PACK,NaN,NaN,NaN,NaN,NaN,NaN
503,030299,NBC BIO WHITE FACIAL FOAM 100 ML,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_match_pi[data_match_pi['Nama Produk (PI)'] == "VIOSTIN FC CAPLET 10*5`S (CATCHCOVER)"]

,Nama Produk (PI),Kata Pertama,Nama Produk (IMS),Skor Kemiripan
186,VIOSTIN FC CAPLET 10*5`S (CATCHCOVER),VIOSTIN,viostin caplet fc 50 strp,52.6


In [ ]:
import os

file_name = f"Database PI.xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
database_match_pi.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/Database PI.xlsx
